In [1]:
import json

import numpy as np
import pandas as pd
from MDAnalysis import Universe
from MDAnalysis.analysis.hydrogenbonds import HydrogenBondAnalysis

from paprika.io import NumpyEncoder

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


## Init

In [2]:
host_resname = "CB7"
guest_resname = "AMT"
solvent_resname = "WAT"

results = {
    "bound": {"time": None, "H-G": None, "H-W": None, "G-W": None, "W-W": None},
    "unbound": {"time": None, "H-W": None, "G-W": None, "W-W": None},
}

### HBonds criteria

In [3]:
distance_cutoff = 3.0
angle_cutoff = 150

## Load Trajectories

In [4]:
universe_bound = Universe(
    f"../../simulations/a000/{host_resname}-{guest_resname}-sol.prmtop",
    "../../simulations/a000/production.dcd",
    in_memory=False,
)

/home/jsetiadi/anaconda3/envs/paprika-dev/lib/python3.8/site-packages/MDAnalysis/topology/TOPParser.py:295: UserWarning: Unknown ATOMIC_NUMBER value found for some atoms, these have been given an empty element record. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(msg)


In [5]:
universe_unbound = Universe(
    f"../../simulations/r014/{host_resname}-{guest_resname}-sol.prmtop",
    "../../simulations/r014/production.dcd",
    in_memory=False,
)

## HBonds in complex state

In [6]:
hbonds_bound_HG = HydrogenBondAnalysis(
    universe=universe_bound,
    between=[
        [f"resname {host_resname}", f"resname {guest_resname}"],
    ],
    d_a_cutoff=distance_cutoff,
    d_h_a_angle_cutoff=angle_cutoff,
    update_selections=False,
)
hbonds_bound_HG.run(verbose=True)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [7]:
hbonds_bound_HW = HydrogenBondAnalysis(
    universe=universe_bound,
    between=[
        [f"resname {host_resname}", f"resname {solvent_resname}"],
    ],
    d_a_cutoff=distance_cutoff,
    d_h_a_angle_cutoff=angle_cutoff,
    update_selections=False,
)
hbonds_bound_HW.run(verbose=True)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [8]:
hbonds_bound_GW = HydrogenBondAnalysis(
    universe=universe_bound,
    between=[
        [f"resname {guest_resname}", f"resname {solvent_resname}"],
    ],
    d_a_cutoff=distance_cutoff,
    d_h_a_angle_cutoff=angle_cutoff,
    update_selections=False,
)
hbonds_bound_GW.run(verbose=True)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
hbonds_bound_WW = HydrogenBondAnalysis(
    universe=universe_bound,
    between=[
        [f"resname {solvent_resname}", f"resname {solvent_resname}"],
    ],
    d_a_cutoff=distance_cutoff,
    d_h_a_angle_cutoff=angle_cutoff,
    update_selections=False,
)
hbonds_bound_WW.run(verbose=True)

  0%|          | 0/100000 [00:00<?, ?it/s]

## HBonds in unbound state

In [ ]:
hbonds_unbound_HW = HydrogenBondAnalysis(
    universe=universe_unbound,
    between=[
        [f"resname {host_resname}", f"resname {solvent_resname}"],
    ],
    d_a_cutoff=distance_cutoff,
    d_h_a_angle_cutoff=angle_cutoff,
    update_selections=False,
)
hbonds_unbound_HW.run(verbose=True)

In [ ]:
hbonds_unbound_GW = HydrogenBondAnalysis(
    universe=universe_unbound,
    between=[
        [f"resname {guest_resname}", f"resname {solvent_resname}"],
    ],
    d_a_cutoff=distance_cutoff,
    d_h_a_angle_cutoff=angle_cutoff,
    update_selections=False,
)
hbonds_unbound_GW.run(verbose=True)

In [ ]:
hbonds_unbound_WW = HydrogenBondAnalysis(
    universe=universe_unbound,
    between=[
        [f"resname {solvent_resname}", f"resname {solvent_resname}"],
    ],
    d_a_cutoff=distance_cutoff,
    d_h_a_angle_cutoff=angle_cutoff,
    update_selections=False,
)
hbonds_unbound_WW.run(verbose=True)

### Results

In [ ]:
results["bound"]["H-G"] = hbonds_bound_HG.count_by_time()
results["bound"]["H-W"] = hbonds_bound_HW.count_by_time()
results["bound"]["G-W"] = hbonds_bound_GW.count_by_time()
results["bound"]["W-W"] = hbonds_bound_WW.count_by_time()

results["unbound"]["H-W"] = hbonds_unbound_HW.count_by_time()
results["unbound"]["G-W"] = hbonds_unbound_GW.count_by_time()
results["unbound"]["W-W"] = hbonds_unbound_WW.count_by_time()


results["bound"]["time"] = (
    np.arange(0, len(results["bound"]["H-G"]) + 1, 1) * universe_bound.trajectory.dt
)

results["unbound"]["time"] = (
    np.arange(0, len(results["unbound"]["H-W"]) + 1, 1) * universe_unbound.trajectory.dt
)

In [ ]:
data = [
    {
        "H-G (bound)": results["bound"]["H-G"].mean(),
        "H-W (bound)": results["bound"]["H-W"].mean(),
        "G-W (bound)": results["bound"]["G-W"].mean(),
        "W-W (bound)": results["bound"]["W-W"].mean(),
        "H-W (unbound)": results["unbound"]["H-W"].mean(),
        "G-W (unbound)": results["unbound"]["G-W"].mean(),
        "W-W (unbound)": results["unbound"]["W-W"].mean(),
    }
]

df = pd.DataFrame(data)
df

### Save results to file

In [ ]:
df.to_csv("hbonds_results.csv")

with open("hbonds_results.json", "w") as f:
    dumped = json.dumps(results, cls=NumpyEncoder)
    f.write(dumped)